# Prompt Engineering : Advanced Prompting avec OpenAI

Dans ce notebook, nous allons tester différentes techniques avancées de **prompt engineering**:
- **Zero-shot prompting**
- **Few-shot prompting**
- **Chain-of-thought** (CoT)
- **Self-refine** (ou auto-amélioration)

Nous utiliserons la **nouvelle API** de la bibliothèque `openai` (>=1.0.0) via la classe `OpenAI` et ses méthodes de chat (`client.chat.completions.create`).


In [2]:
# ============================
# Cellule 1 : Installation
# ============================

%pip install openai tiktoken python-dotenv
# Remarque : Aucune fin de ligne en commentaire pour éviter l'erreur


Note: you may need to restart the kernel to use updated packages.


In [4]:
# ============================
# Cellule 2 : Configuration
# ============================

import os
from dotenv import load_dotenv

load_dotenv()

# On suppose que ton .env contient :
# OPENAI_API_KEY=sk-xxxxxx
# (ou autre variable si tu utilises Azure)
#
# Récupère la clé d'API
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Clé API introuvable. Vérifie ton fichier .env.")


In [5]:
# ============================
# Cellule 3 : Client OpenAI
# ============================

import openai
from openai import OpenAI

# Pour l'exemple, on définit le modèle par défaut
MODEL_NAME = "gpt-4o-mini"

# Instanciation du client
client = OpenAI(
    api_key=api_key,
    # Tu peux configurer d'autres options si besoin
)

print("Client OpenAI initialisé avec succès !")


Client OpenAI initialisé avec succès !


### Rappel des différences entre Zero-shot, Few-shot, Chain-of-thought et Self-refine

1. **Zero-shot Prompting**  
   - Aucune instruction ou exemple préalable (à part la demande de l'utilisateur).  
   - Simple et direct, mais parfois moins précis ou cohérent.

2. **Few-shot Prompting**  
   - Fournir quelques exemples “input → output” pour guider la réponse.  
   - Permet de **spécifier le format**, le style, ou le contenu souhaité.  
   - Améliore significativement la qualité des réponses sur des tâches complexes.

3. **Chain-of-thought (CoT)**  
   - On **incite** le modèle à détailler son raisonnement étape par étape.  
   - Souvent utile pour des questions de logique, mathématiques, programmation ou raisonnement complexe.  
   - Peut **augmenter** la cohérence et la justesse de la réponse (mais attention à ne pas divulguer ces “étapes” si elles sont confidentielles).

4. **Self-refine**  
   - Demander au modèle de s’auto-critiquer puis de proposer une réponse améliorée.  
   - Mise en œuvre en plusieurs appels (réponse initiale, re-demande d’analyse, ré-énoncé final).  
   - Intéressant pour du code, des textes longs, ou des situations nécessitant un contrôle qualité.

---


In [6]:
# ============================
# Cellule 5 : Zero-shot
# ============================

prompt_1 = "Donne-moi 3 idées de recettes végétariennes à base de tomates."
response_1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_1}
    ],
    # Contrôle du style
    max_tokens=200,
    temperature=0.7  # plus la température est haute, plus c'est créatif
)

print("=== Zero-shot Prompt ===")
print(f"Prompt: {prompt_1}\n")
print("Réponse du modèle :\n")
print(response_1.choices[0].message.content)


=== Zero-shot Prompt ===
Prompt: Donne-moi 3 idées de recettes végétariennes à base de tomates.

Réponse du modèle :

Voici trois délicieuses idées de recettes végétariennes à base de tomates :

### 1. **Tarte tomate et basilic**
**Ingrédients :**
- Pâte brisée
- Tomates mûres
- Fromage de chèvre ou mozzarella
- Basilic frais
- Huile d'olive
- Sel et poivre

**Préparation :**
1. Préchauffez le four à 180°C.
2. Étalez la pâte brisée dans un moule à tarte et piquez le fond avec une fourchette.
3. Disposez des rondelles de tomates sur la pâte.
4. Ajoutez des tranches de fromage de chèvre ou de mozzarella par-dessus.
5. Assaisonnez avec de l'huile d'olive, du sel, du poivre et du basilic frais.
6. Enfournez pendant 30 à 35 minutes, jusqu'à ce que la pâte soit dorée et le fromage fondu


Ici, pas d’exemples ni d’instructions détaillées, on se contente d’un prompt direct.



###  Exemple Few-shot prompting (Code)

In [7]:
# ============================
# Cellule X (NOUVELLE) : Few-shot supplémentaire
# ============================

few_shot_prompt_2 = """
Tu es un assistant spécialisé en rédaction d'e-mails professionnels.
Voici quelques exemples de style :

Exemple 1:
Q: Rédige un e-mail pour informer un client d'un retard de livraison
A: 
Sujet: Information concernant le retard de votre livraison

Bonjour [Nom du Client],

Nous tenions à vous informer que votre commande #1234 a pris du retard...
[...suite du mail...]

Exemple 2:
Q: Envoie un e-mail de remerciement pour un entretien d'embauche
A:
Sujet: Remerciements suite à notre entretien

Bonjour [Nom du Contact],

Je tiens à vous remercier pour le temps que vous m'avez accordé...
[...suite du mail...]

Maintenant, voici ma demande:

Q: Écris un e-mail pour informer un collaborateur d'un changement de planning et l'inviter à une réunion de suivi.
A:
"""

response_few_shot_2 = client.chat.completions.create(
    model=MODEL_NAME,  # ex. "gpt-4o-mini"
    messages=[
        {"role": "user", "content": few_shot_prompt_2}
    ],
    max_tokens=300,
    temperature=0.6
)

print("=== Exemple Few-shot (e-mail professionnel) ===")
print(response_few_shot_2.choices[0].message.content)


=== Exemple Few-shot (e-mail professionnel) ===
Sujet: Changement de planning et invitation à une réunion de suivi

Bonjour [Nom du Collaborateur],

Je souhaite vous informer d'un changement concernant notre planning initial. En raison de [raison du changement], nous devons ajuster certaines échéances.

Pour discuter des impacts de ces modifications et assurer une bonne coordination, je vous invite à une réunion de suivi qui se tiendra le [date] à [heure]. Nous pourrons ainsi aborder les nouvelles échéances et répondre à toutes vos questions.

Merci de me confirmer votre disponibilité pour cette réunion.

Cordialement,

[Votre Nom]  
[Votre Poste]  
[Votre Entreprise]  
[Votre Numéro de Téléphone]  
[Votre Adresse E-mail]


Ici, nous donnons au modèle **deux exemples** de questions/réponses avant la **véritable question**. Cela oriente le style et le contexte.



###  7 : Exemple Chain-of-thought (Code)

On va demander un **calcul** simple, en guidant le modèle à réfléchir pas à pas :



- **Masquer le raisonnement si nécessaire** :  
  Parfois, on ne souhaite pas afficher au client final les étapes du raisonnement. Il existe des techniques pour “cacher” ce raisonnement ou n’afficher qu’un résumé. Par exemple :  
  1. Dans le prompt, on peut demander : “Don’t reveal your chain-of-thought. Provide only the final concise answer to the user.”  
  2. Ou effectuer un 2e appel API où l’on transmet l’enchaînement de pensées, mais on n'affiche que la conclusion.

- **Adapter la température** :  
  Pour un problème logique ou mathématique, une température trop élevée peut introduire des dérives ou des incohérences. Une température entre 0.0 et 0.3 est souvent recommandée pour les questions de calcul.

- **Chain-of-thought partiel** :  
  On peut encourager un raisonnement **intermédiaire** (quelques étapes clés) au lieu d’un raisonnement hyper détaillé, afin d’éviter que le texte devienne trop long ou difficile à comprendre.

- **Expliquer la démarche** :  
  On peut terminer par un résumé du raisonnement en 2-3 phrases, pour rendre la solution plus lisible.


In [8]:
# ============================
# Cellule 7 : Chain-of-thought
# ============================

cot_prompt = """
Alice a 5 pommes, elle en jette 2, puis elle en donne 1 à Bob.
Bob lui rend ensuite 1 pomme.
Combien de pommes Alice a-t-elle à la fin ?

Explique ton raisonnement étape par étape, puis donne la réponse finale.
"""

response_3 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    max_tokens=200,
    temperature=0.2  # on réduit la température pour moins de fantaisie
)

print("=== Chain-of-thought Prompt ===")
print("Réponse du modèle (avec raisonnement) :\n")
print(response_3.choices[0].message.content)


=== Chain-of-thought Prompt ===
Réponse du modèle (avec raisonnement) :

Pour résoudre ce problème, nous allons suivre les étapes une par une.

1. **Alice commence avec 5 pommes.**
   - Pommes d'Alice : 5

2. **Elle en jette 2.**
   - Pommes d'Alice après avoir jeté 2 : 5 - 2 = 3

3. **Elle en donne 1 à Bob.**
   - Pommes d'Alice après avoir donné 1 à Bob : 3 - 1 = 2

4. **Bob lui rend ensuite 1 pomme.**
   - Pommes d'Alice après que Bob lui a rendu 1 : 2 + 1 = 3

Donc, à la fin, Alice a **3 pommes**.


On demande explicitement « explique ton raisonnement ». Cela **n’oblige** pas le modèle à le faire, mais en pratique, GPT-4o-mini (ou tout modèle qui gère le CoT) fournit souvent une solution pas-à-pas.

---

###  8 : Exemple Self-refine (Code)

L’idée : on fait **une première demande** (première réponse) et ensuite **on redemande** au modèle de s’auto-corriger.

#### 8a. Premier prompt

In [9]:
# ============================
# Cellule 8a : Self-refine Step 1
# ============================

prompt_sr1 = """
Ecris une courte fonction Python pour calculer la somme d'une liste. 
Ajoute un bug volontaire dans le code. 
"""

response_sr1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr1}],
    max_tokens=300
)

buggy_code = response_sr1.choices[0].message.content

print("=== Self-refine (1) : Code buggy ===\n")
print(buggy_code)


=== Self-refine (1) : Code buggy ===

Voici une fonction Python qui calcule la somme des éléments d'une liste, mais avec un bug volontaire :

```python
def somme_liste(liste):
    total = 0
    for i in range(len(liste)):
        total += liste[i]
    return total + 1  # Bug volontaire : on ajoute 1 à la somme finale

# Exemple d'utilisation
ma_liste = [1, 2, 3, 4]
resultat = somme_liste(ma_liste)
print(resultat)  # devrait afficher 10, mais affichera 11 à cause du bug
```

Dans cette fonction, le bug volontaire est que nous ajoutons 1 à la somme finale, ce qui fausse le résultat.


#### 8b. Self-critique et amélioration

In [10]:
# ============================
# Cellule 8b : Self-refine Step 2
# ============================

prompt_sr2 = f"""
Voici un code Python qui contient un bug:

{buggy_code}

Peux-tu l'analyser, détecter le bug, proposer un correctif et une version améliorée du code ? 
Explique la correction.
"""

response_sr2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr2}],
    max_tokens=400,
    temperature=0.3
)

print("=== Self-refine (2) : Correction ===\n")
print(response_sr2.choices[0].message.content)


=== Self-refine (2) : Correction ===

Bien sûr ! Analysons le code que vous avez fourni.

### Analyse du code

La fonction `somme_liste` est censée calculer la somme des éléments d'une liste. Cependant, il y a un bug volontaire où `1` est ajouté à la somme finale, ce qui fausse le résultat. Par exemple, pour la liste `[1, 2, 3, 4]`, la somme correcte devrait être `10`, mais à cause de l'ajout de `1`, le résultat affiché est `11`.

### Correction du bug

Pour corriger ce bug, il suffit de supprimer l'ajout de `1` à la somme finale. Voici la version corrigée de la fonction :

```python
def somme_liste(liste):
    total = 0
    for i in range(len(liste)):
        total += liste[i]
    return total  # Suppression de l'ajout de 1

# Exemple d'utilisation
ma_liste = [1, 2, 3, 4]
resultat = somme_liste(ma_liste)
print(resultat)  # Affichera maintenant 10
```

### Version améliorée du code

Nous pouvons également améliorer le code en utilisant la fonction intégrée `sum()`, qui est plus concise

In [11]:
# ============================
# Cellule X (NOUVELLE) : Self-refine avec developer role
# ============================

messages_sr = [
    {
        "role": "developer",
        "content": (
            "You are a self-improving coding assistant. Whenever you provide code, "
            "you will automatically search for potential bugs or improvements "
            "and refine your output."
        )
    },
    {
        "role": "user",
        "content": (
            "Écris une fonction Python qui calcule la factorielle d'un nombre entier. "
            "Ensuite, relis-toi et corrige d'éventuels bugs."
        )
    }
]

response_self_refine = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages_sr,
    max_tokens=300,
    temperature=0.3
)

print("=== Self-refine avec 'developer role' ===")
print(response_self_refine.choices[0].message.content)


=== Self-refine avec 'developer role' ===
Voici une fonction Python qui calcule la factorielle d'un nombre entier. Je vais également vérifier et corriger d'éventuels bugs.

```python
def factorielle(n):
    """Calcule la factorielle d'un nombre entier n."""
    if not isinstance(n, int):
        raise ValueError("L'argument doit être un entier.")
    if n < 0:
        raise ValueError("La factorielle n'est pas définie pour les nombres négatifs.")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result
```

### Améliorations et vérifications :

1. **Validation de l'entrée** : La fonction vérifie maintenant si l'argument est un entier et s'il est non négatif.
2. **Utilisation d'une boucle** : La méthode itérative est utilisée pour éviter les problèmes de dépassement de pile qui peuvent survenir avec la récursivité pour de grandes valeurs de `n`.
3. **Documentation** : Une docstring a été ajoutée pour expliquer ce que fa

Ici, on utilise la première réponse pour nourrir le second prompt, demandant au modèle de **critiquer** et **améliorer** la réponse initiale.

---

### Cellule 9 : Interactive Prompt (Code)

Enfin, on peut proposer une **cellule interactive** : l’utilisateur peut saisir un prompt, et on envoie la requête au modèle :

In [12]:
# ============================
# Cellule 9 : Prompt interactif
# ============================

while True:
    user_input = input("Tape ton prompt ('exit' pour quitter) : ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("Fin de l'interaction.")
        break
    
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role":"user","content":user_input}],
        max_tokens=200,
        temperature=0.7
    )
    
    print("\n=== Réponse du modèle ===")
    print(resp.choices[0].message.content)
    print("---------------------------------------------------\n")



=== Réponse du modèle ===
Aujourd'hui, nous sommes le 27 octobre 2023.
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Salut ! Comment puis-je t'aider aujourd'hui ?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you today?
---------------------------------------------------


=== Réponse du modèle ===
Hello! How can I assist you toda

KeyboardInterrupt: Interrupted by user

In [ ]:
# ============================
# Cellule 9 : Prompt interactif avec mémoire de chat
# ============================
user_input = "";
current_messages=[]

while True:
    user_input = input("Tape ton prompt ('exit' pour quitter) : ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("Fin de l'interaction.")
        break
    print("\n=== message de l'utilisateur ===")
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages = current_messages + [{"role":"user","content":user_input}],
        max_tokens=200,
        temperature=0.7
    )
    
    print("\n=== Réponse du modèle ===")
    assistant_message = resp.choices[0].message.content
    print(assistant_message)
    print("---------------------------------------------------\n")
    current_messages.append({"role":"assistant","content":assistant_message})


Fin de l'interaction.


Maintenant, tu peux saisir n’importe quel prompt, et tu verras la réponse du modèle.  
Tape `exit` pour quitter la boucle.

----

## Conclusion et Ressources Supplémentaires

Dans ce notebook, nous avons approfondi diverses techniques de **prompt engineering** : 
- Zero-shot  
- Few-shot  
- Chain-of-thought  
- Self-refine  
- Interactions multi-messages (avec les rôles `system`, `developer`, `user`, `assistant`)  

### Ressources conseillées
- [**OpenAI Cookbook**](https://github.com/openai/openai-cookbook) : Recettes et astuces pour résoudre des problèmes concrets (prompt engineering, RAG, etc.).  
- [**Prompt Engineering Guide**](https://www.promptingguide.ai/) : Conseils de rédaction de prompts, cas d’usages, bonnes pratiques.  
- [**Chaine d’outils** (LangChain, LlamaIndex, etc.)](https://github.com/hwchase17/langchain) : Facilite la création de pipelines complexes (RAG, function calling, mémoire de conversation).  

> **Idées d’exercices**  
> 1. Adapter la technique Few-shot à d’autres cas (e.g., Q&R sur la finance, la santé ou le marketing).  
> 2. Utiliser la Self-refine pour générer un texte marketing, puis le réécrire en style “plus formel” ou “plus humoristique”.  
> 3. Tester la **combinaison** de techniques : un prompt Few-shot + Chain-of-thought + un re-run Self-refine.

---

Merci d'avoir suivi ce notebook sur le **prompt engineering avancé** !
